In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import plotly.express as px
import matplotlib as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
link = "/content/drive/MyDrive/train.csv"
df = pd.read_csv(link)

In [ ]:
display(df.head())

,textID,text,selected_text,sentiment
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative
2,088c60f138,my boss is bullying me...,bullying me,negative
3,9642c003ef,what interview! leave me alone,leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative


In [ ]:
df.shape

(27480, 4)

#Méthodologie

1. Train test split

2. Fit et transform du Vectorizer sur le train set
3. Fit et score du classifier sur le train set
4. Transform (sans fit !) du Vectorizer sur le test set
5. Predict et score du classifier sur le test set

#Challenge

1. Importe cet ensemble de données de tweets dans un DataFrame

2. Ne garde que les tweets positifs et négatifs (tu excluras donc les neutral). Quel est le pourcentage de tweets positifs/négatifs ?

In [ ]:
df_filtered = df[df['sentiment'] != 'neutral']

In [ ]:
print(((df_filtered["sentiment"].value_counts() / len(df_filtered["sentiment"])) * 100).round(2))

sentiment
positive    52.45
negative    47.55
Name: count, dtype: float64


3. Copie la colonne text dans une Série X, et la colonne sentiment dans une Série y. Applique un train test split avec le random_state = 32 et un train_size de 0.75

In [ ]:
X = df_filtered["text"]
y = df_filtered["sentiment"]

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, random_state=32)

print(len(X_train))
print(len(X_test))
print(len(y_train))
print(len(y_test))

12272
4091
12272
4091


4. Crée un modèle vectorizer avec scikit-learn en utilisant la méthode Countvectorizer. Entraîne ton modèle sur X_train, puis crée une matrice de features X_train_CV. Crée la matrice X_test_CV sans ré-entraîner le modèle. Le format de la matrice X_test_CV doit être 4091x15806 avec 44633 stored elements

In [ ]:
# entrainment sur X
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
vectorizer.fit(X_train)

CountVectorizer()

In [ ]:
X_train_CV = vectorizer.transform(X_train)
X_train_CV

<12272x15806 sparse matrix of type '<class 'numpy.int64'>'
	with 144578 stored elements in Compressed Sparse Row format>

In [ ]:
X_test_CV = vectorizer.transform(X_test)
X_test_CV

<4091x15806 sparse matrix of type '<class 'numpy.int64'>'
	with 44633 stored elements in Compressed Sparse Row format>

5. Entraîne maintenant une régression logistique avec les paramètres par défaut. Tu devrais obtenir les résultats suivants : 0.966pour le test d'entraînement, et 0.877 pour l'ensemble de test

In [ ]:
X_train_M = X_train_CV.toarray()
X_test_M = X_test_CV.toarray()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model_lg = LogisticRegression().fit(X_train_M, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [ ]:
# evaluation du modèle sur le set d'entrainement

y_pred_train = model_lg.predict(X_train_M)
accuracy_train = accuracy_score(y_train, y_pred_train)
print(f"Précision sur l'ensemble d'entraînement: {accuracy_train:.3f}")

Précision sur l'ensemble d'entraînement: 0.966


In [ ]:
# evaluation du modèle sur le set de test

y_pred_test = model_lg.predict(X_test_M)
accuracy_test = accuracy_score(y_test, y_pred_test)
print(f"Précision sur l'ensemble de test: {accuracy_test:.3f}")

Précision sur l'ensemble de test: 0.877


6. Étape bonus : essaye d'afficher 10 tweets qui ont été mal prédits (faux positifs ou faux négatifs). Aurais-tu fait mieux que l'algorithme ?

In [ ]:
from sklearn.metrics import confusion_matrix

pd.DataFrame(data = confusion_matrix(y_true = y_test, y_pred = model_lg.predict(X_test_M)),
             index = model_lg.classes_ + " ACTUAL",
             columns = model_lg.classes_ + " PREDICTED")

,negative PREDICTED,positive PREDICTED
negative ACTUAL,1700,235
positive ACTUAL,267,1889


In [ ]:
# Afficher 10 tweets mal prédits

# Récupérer les labels actual et predicted
actual_labels = y_test.to_numpy()
predicted_labels = y_pred

# Crée un dataframe qui rassemble les tweets et les labels
tweet_data = pd.DataFrame({'tweet': X_test, 'actual_label': actual_labels, 'predicted_label': predicted_labels})

# Extrait les tweets mal prédits
misclassified_tweets = tweet_data[tweet_data['actual_label'] != tweet_data['predicted_label']]

# les affiche
print("10 Tweets mal prédits:")
for i in range(10):
    tweet = misclassified_tweets.iloc[i]['tweet']
    actual_label = misclassified_tweets.iloc[i]['actual_label']
    predicted_label = misclassified_tweets.iloc[i]['predicted_label']
    print(f"#{i+1}: Tweet: {tweet}")
    print(f"    Label réel: {actual_label}")
    print(f"    Label prédit: {predicted_label}")
    print()

10 Tweets mal prédits:
#1: Tweet: HollowbabesHere comes the utter shite #bgt <I completely agree
    Label réel: negative
    Label prédit: positive

#2: Tweet:  SUFFICATION NO BREATHING. It`s okay. There`ll be more. You`re invited to mine, but I can`t promise fun times.  *Jinx
    Label réel: negative
    Label prédit: positive

#3: Tweet: i wanna vote for Miley Cyrus for the mtv movie awards..but i don`t know where i could  somebody could send me a link? thaank you <3
    Label réel: positive
    Label prédit: negative

#4: Tweet: I love music so much that i`ve gone through pain to play :S my sides of my fingers now are peeling and have blisters from playing so much
    Label réel: negative
    Label prédit: positive

#5: Tweet: I can only message those who message me, if we`re fwends...so those that want replies..follow me.  hmm..that sounds funny..
    Label réel: positive
    Label prédit: negative

#6: Tweet: wish I could feel no pain (8)  but it`s ok, at least they like Brazil!


Resterait à améliorer le modèle

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model_2 = LogisticRegression(class_weight = {"negative": 0.2, "positive": 0.5}).fit(X_train_M, y_train)

In [ ]:
# evaluation du modèle sur le set d'entrainement

y_pred_train = model_2.predict(X_train_M)
accuracy_train = accuracy_score(y_train, y_pred_train)
print(f"Précision sur l'ensemble d'entraînement: {accuracy_train:.3f}")

Précision sur l'ensemble d'entraînement: 0.907


In [ ]:
# evaluation du modèle sur le set de test

y_pred_test = model_2.predict(X_test_M)
accuracy_test = accuracy_score(y_test, y_pred_test)
print(f"Précision sur l'ensemble de test: {accuracy_test:.3f}")

Précision sur l'ensemble de test: 0.846


In [ ]:
from sklearn.metrics import confusion_matrix

pd.DataFrame(data = confusion_matrix(y_true = y_test, y_pred = model_2.predict(X_test_M)),
             index = model_lg.classes_ + " ACTUAL",
             columns = model_lg.classes_ + " PREDICTED")

,negative PREDICTED,positive PREDICTED
negative ACTUAL,1426,509
positive ACTUAL,119,2037


Ici il faudrait sans doute que je fasse une fonction qui permet de faire varier les class_weight pour voir les différentes combinaisons et les scores et résultats associés